In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import seaborn as sns
#import matplotlib.pyplot as plt
#%matplotlib inline

In [ ]:
#settimg path for the directory
import os
os.chdir("C:/Users/Lalith/Downloads/P1 Data")

In [ ]:
#loding the data sets from the directory
filepath1='C:/Users/Lalith/Downloads/P1 Data/Consumer_Complaints_train.csv'
pm15_train=pd.read_csv(filepath1)
filepath2='C:/Users/Lalith/Downloads/P1 Data/Consumer_Complaints_test_share.csv'
pm15_test=pd.read_csv(filepath2)


In [ ]:
#renaming the column name
pm15_train=pm15_train.rename(columns={'Consumer disputed?':'consumerdisputed'})


In [ ]:
#printing the head for train dataset
pm15_test.head()

In [ ]:
#printing the head for the test data set
pm15_train.head()

In [ ]:
#droping insignificant columns, and the column which we are going to take as target\dependent variable
X_train=pm15_train.drop(['Complaint ID','consumerdisputed','Company','ZIP code'],axis=1)

X_test=pm15_test.drop(['Complaint ID','ZIP code','Company'],axis=1)

In [ ]:
#taking dependent variable and converting it's values into machine readble formet
y_train=pm15_train['consumerdisputed'].map({'Yes':1,'No':0,})


In [ ]:
#printing lcolumn length of the train and test data sets
print(len(pm15_train.columns))
print(len(pm15_test.columns))

In [ ]:
#printing the columns present in the training set
X_train.columns

In [ ]:
#printing the training set to know the object type columns
X_train.dtypes=='object'



In [ ]:
#converting the the date column  which is in object type into datetype  for training data set
X_train['Date received']=pd.to_datetime(X_train['Date received'])
X_train['Date sent to company']=pd.to_datetime(X_train['Date sent to company'])

In [ ]:
#converting the the date column  which is in object type into datetype  for test data set

X_test['Date received']=pd.to_datetime(X_test['Date received'])
X_test['Date sent to company']=pd.to_datetime(X_test['Date sent to company'])
 

In [ ]:
# we are taking the diference of the the two Date sent to company,Date received to train the model because individual date columns has no meaningfull insight for the ata set 
X_train['day_diff']=pd.to_numeric(X_train['Date sent to company']-X_train['Date received'])
X_train.drop(['Date sent to company','Date received'],axis=1,inplace=True)


In [ ]:
X_test['day_diff']=pd.to_numeric(X_test['Date sent to company']-X_test['Date received'])
X_test.drop(['Date sent to company','Date received'],axis=1,inplace=True)


In [ ]:
# importing labelencoder to preocess the Consumer complaint narrative'

from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
X_train['Consumer complaint narrative']=pd.to_numeric(X_train['Consumer complaint narrative'],errors='coerce')
X_train['Consumer complaint narrative']=labelencoder.fit_transform(X_train['Consumer complaint narrative'])


In [ ]:

X_test['Consumer complaint narrative']=pd.to_numeric(X_test['Consumer complaint narrative'],errors='coerce')
X_test['Consumer complaint narrative']=labelencoder.fit_transform(X_test['Consumer complaint narrative'])


In [ ]:
#printing training dataset head to see the alinghment
X_train.head(3)

In [ ]:
X_test.head(2)

In [ ]:
#checking the column length for both training and test data to ensure column length was matched
print(X_test.shape[1])
print(X_train.shape[1])

In [ ]:
#spliting the traning data set into objecttype and numbertype for further processing  
numcol=X_train.select_dtypes(include=[np.number])
catcol=X_train.select_dtypes(include=[np.object])

In [ ]:
#spliting the test data set into objecttype and numbertype for further processing  

numcol1=X_test.select_dtypes(include=[np.number])
catcol1=X_test.select_dtypes(include=[np.object])


In [ ]:
#getting dummies for the obect type columns of training data set
catcol=pd.get_dummies(catcol)

In [ ]:
#getting dummies for the obect type columns of test data set

catcol1=pd.get_dummies(catcol1)


In [ ]:
#scaling the number type columns to normalize the data of training data set
numcol=preprocessing.scale(numcol)

In [ ]:
#scaling the number type columns to normalize the data of test data set

numcol1=preprocessing.scale(numcol1)

In [ ]:
#converting the array type numeric columns into dataframe to be concat with categorical types a
numcol=pd.DataFrame(numcol)#training data set 
numcol1=pd.DataFrame(numcol1)#test data set

In [ ]:
#concatinating the two data frames into single dataframe to be used as X_train 
X_train=pd.concat([catcol,numcol],axis=1)#we are concating with reference to columns by taking axis=1, if 0 it will do row wise concatination

In [ ]:
X_train.head(2)

In [ ]:
#concatinating the two data frames into single dataframe to be used as X_test

X_test=pd.concat([catcol1,numcol1],axis=1)

In [ ]:
X_test.head(2)

In [ ]:
# importing train_test_split function to split the data sets and 
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train,random_state=0)


In [ ]:
                                                #MODEL:1
#importing logistic regression model and fiting the model  
from sklearn.linear_model import LogisticRegression
clf=LogisticRegression()
clf.fit(X_train,y_train)


In [ ]:
# predicting the test data  
predictions=clf.predict(X_test)

In [ ]:
logreg=clf.predict_proba(X_test)[:,1]
print(logreg)

In [ ]:
def pridict_threshold (clf,X_test,threshold):
    return np.where(logreg>threshold,1,0)


In [ ]:
for thr in np.arange(0,.3,.1):
    predictions=pridict_threshold(clf,X_test,thr)
    print(confusion_matrix(y_test,predictions))   

In [ ]:
# importing metrics to find accurecy
from sklearn.metrics import accuracy_score,confusion_matrix
cm = confusion_matrix(y_test,predictions)
print(cm)



In [ ]:
from sklearn.metrics import roc_auc_score
AUC=roc_auc_score(y_test,predictions)
print(AUC)

In [ ]:
#printing accurecy
acc=accuracy_score(y_test,predictions)
print(acc)

In [ ]:
predictions=np.where(clf.predict(X_test)==1,"Yes","No")


In [ ]:
submission=pd.DataFrame(list(zip(pm15_test['Complaint ID'],list(prediction))),
                       columns=['Complaint ID','consumerdisputed'])
